# 순환 신경망 (RNN)

## 언어 모델

CBOW의 한계
* 맥락의 크기 외부에 있는 단어의 정보는 무시된다.
* 맥락의 어순을 고려하지 않는다.
* 사실 word2vec은 단어의 분산표현을 얻으려는 목적, 언어모델로 사용 X

순환 신경망(RNN): 맥락이 아무리 길더라도 그 정보를 기억한다.

$\textbf{h}_t = tanh(\textbf{h}_{t-1}\textbf{W}_h+\textbf{x}_t\textbf{W}_x + \textbf{b})$
* 은닉 벡터 $\textbf{h}_t$는 (1)다른 계층으로, (2)자기자신을 향해 양방향으로 출력된다.

## BPTT

BPTT(backpropagation through time): RNN에서 사용되는 역전파법, 시간 방향으로 펼쳐서 생각해 보자
* 문제점: 큰 시계열 데이터 학습 시 많은 컴퓨팅자원 사용
* Truncated BPTT: 역전파의 연결을 적당한 길이로 잘라내, 그 단위로 학습 수행 (다른 블록의 데이터는 고려하지 않아도 됨)
* 미니배치 학습의 경우, 각 미니배치의 시작 위치를 offset(맨 처음)으로 옮겨준 후 순서대로 입력하면 됨

# RNN 구현

Time RNN 계층: 순환구조를 펼친 후의 계층들을 하나의 계층으로 간주

In [2]:
# 일반 RNN 계층

import numpy as np

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
    
    def forward(self, x, h_prev):
        # h_prev (x_t-1): (N, H)
        # Wh: (H, H)
        # x (x_t): (N, D)
        # Wx: (D, H)
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tanh(t)
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next):
        Wx, Wh, Wb = self.params
        x, h_prev, h_next = self.cache
        
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matmul(x.T, dt)
        dx = np.matmul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev

In [3]:
# T개의 RNN 계층 (Time RNN 계층)
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None # RNN 계층을 리스트로 저장
         
        self.h, self.dh = None, None
        # 마지막 RNN계층의 은닉 상태 / 하나 앞 블록 은닉상태의 기울기
        self.stateful = stateful # False일 시 은닉상태를 0으로 초기화
        
    def set_state(self, h):
        self.h = h
        
    def reset_state(self):
        self.h = None
        
    def forward(self, xs):
        # xs: T개 분량의 시계열데이터를 모은 것, (N, T, D)
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape
        
        self.layers = []
        hs = np.empty((N, T, H), dtype='f')
        
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')
        # 은닉상태는 영행렬로 초기화
        
        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)
        
        return hs

    def backward(self, dhs):
        # dhs: 출력층에서 전해지는 기울기
        # dxs: 하류로 내보내는 기울기
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape
        
        dxs = np.empty((N, T, D), dtpye='f')
        dh = 0
        grads = [0, 0, 0]
        
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
            
        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
            # Time RNN계층의 최종 가중치 기울기는, 각 RNN 계층의 가중치 기울기를 모두 더한 것
        self.dh = dh
        
        return dxs    
            
        

# 시계열 데이터 처리 계층 구현

$\bf{w_t}$ -> embedding -> RNN -> affine -> softmax 순으로 쌓은 신경망 만들기
* 문장 sequence를 입력할 때, 각 token에 대해 다음에 올 token을 예측한다.
* Time Embedding, Time Affine, Time Softmax with Loss 구조를 구현할 필요가 있다.
* 최종 손실은 $L = \frac{1}{T}(L_0 + L_1 + ... + L_{t-1})$

`simple_rnnlm.py`을 보세요~

## 언어 모델의 평가

perplexity(혼란도)
* 확률의 역수, 작을수록 좋음
* 분기 수: 다음에 취할 선택사항의 수 (퍼플렉시티가 5: 후보가 아직 5개나 된다)

$ L = -\frac{1}{N}\sum_n\sum_kt_{nk}logy_{nk}$

$ perplexity = e^L$

* $N$: 데이터 총개수, $t_n$: 원핫벡터 정답 레이블, $t_{nk}$: n개째 데이터의 k번째 값, $y_{nk}$: softmax함수의 출력

학습 과정은 `train_custom_loop.py`를 보세요~